#### Gold Layer: Aggregated Salary Data for Reporting
#### This notebook performs aggregation using MySQL tables in the Silver layer and stores the results in the Gold layer (`gold_db.salary_gold`) for Power BI reporting.

#### Aggregations:
- Total and Average Salary by Department
- Total Salary by Employee
- Time-based Salary Trends

In [0]:
# ✅ MySQL Connection Setup
jdbc_url = "jdbc:mysql://vikrant-mysql.mysql.database.azure.com:3306"
username = "vicky"
password = "vikrant@123"
silver_db = "silver_db"
gold_db = "gold_db"

In [0]:
### Read Silver Tables from MySQL
salary_df = spark.read.format("jdbc").option("url", f"{jdbc_url}/{silver_db}") \
    .option("dbtable", "salary_silver") \
    .option("user", username).option("password", password).load()

employee_df = spark.read.format("jdbc").option("url", f"{jdbc_url}/{silver_db}") \
    .option("dbtable", "employee_silver") \
    .option("user", username).option("password", password).load()

department_df = spark.read.format("jdbc").option("url", f"{jdbc_url}/{silver_db}") \
    .option("dbtable", "department_silver") \
    .option("user", username).option("password", password).load()

In [0]:
### Join and Aggregate Data
from pyspark.sql.functions import sum, avg, current_date

joined_df = salary_df.join(employee_df, "employee_id").join(department_df, "department_id")

gold_df = joined_df.groupBy("department_name").\
            agg(sum("salary_amount").alias("total_salary_value"),
            avg("salary_amount").alias("avg_salary")
            ).withColumn("report_date", current_date())

In [0]:
### Write Aggregated Data to Gold Table in MySQL
gold_df.write.format("jdbc").option("url", f"{jdbc_url}/{gold_db}") \
    .option("dbtable", "salary_gold") \
    .option("user", username).option("password", password) \
    .mode("overwrite").save()